## Nonlinear biomechanical model
A biomechanical model simulating the mouse ankle joint with a nonlinear muscle model. Model parameters of the skeletal segment are estimated based on the mouse forelimb. Joint excursion is constrained by exponential elastic stops. Small viscosity is added to improve the stability of the system, which is necessary for the elastic stops to work appropriately. Nonlinear properties of muscle are introduced to prevent the system from drifting when there is an offset between the flexor and extensor activities. 

### Import Libraries

In [1]:
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
device = 'cpu'

### Define muscle model functions 
* FL: force-legnth relationship
* FV: force-velocity relationship
* Fpe: passive elastic force

In [2]:
def FL_function(L):
    beta = 1.55
    omega = 0.75
    rho = 2.12
    
    FL = np.exp(-np.power(abs((np.power(L,beta) - 1)/(omega)),rho));
    return FL

In [3]:
def FV_con_function(L,V):
    Vmax = -9.15*2;
    cv0 = -5.7;
    cv1 = 9.18;
    
    FVcon = (Vmax - V)/(Vmax + (cv0 + cv1*L)*V);
    return FVcon

In [4]:
def FV_ecc_function(L,V):
    av0 = -1.4;
    av1 = 0.0;
    av2 = 0.0;
    bv = 0.72;
    FVecc = (bv - (av0 + av1*L + av2*np.power(L,2))*V)/(bv+V);
    
    return FVecc

In [5]:
def Fpe_function(L,V):
    c1_pe1 = 23.0*6
    k1_pe1 = 0.046
    Lr1_pe1 = 1.4
    eta = 0.0001;
    
    Fpe1 = c1_pe1 * k1_pe1 * np.log(np.exp((L - Lr1_pe1)/(k1_pe1))+1) + eta*V;
    
    return Fpe1